# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [70]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [71]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)


In [72]:
# confirm that our new database was created
db_list = mongo.list_database_names()

if "uk_food" in db_list:
    print("The database 'uk_food' exists.")
else:
    print("The database 'uk_food' does not exist.")

The database 'uk_food' exists.


In [73]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [74]:
# review the collections in our new database
collections = db.list_collection_names()
print(collections)

['establishments']


In [75]:
# review a document in the establishments collection
one_document = db.establishments.find_one()
pprint(one_document)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65691efbc41c4594c0290a68'),
 'geocode': {'latitude': Decimal128('51.083812'),
             'longitude': Decimal128('1.195625')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel':

In [76]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [77]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "RatingValue": "",
    "NewRatingPending": True,
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "ChangesByServerID": 0,
    "Distance": 4623.9723280747176,
    "LocalAuthorityBusinessID": "",
    "RatingDate": "",
    "RatingKey": "",
    "RightToReply": "",
    "SchemeType": "FHRS",
    "links": [],
    "meta": {
        "dataSource": None,
        "extractDate": "0001-01-01T00:00:00",
        "itemCount": 0,
        "pageNumber": 0,
        "pageSize": 0,
        "returncode": None,
        "totalCount": 0,
        "totalPages": 0
    }
}

In [78]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)

In [79]:
# Check that the new restaurant was inserted
restaurant_check = establishments.find_one({"BusinessName": "Penang Flavours"})
if restaurant_check:
    print(f"Restaurant 'Penang Flavours' found in the database.")
else:
    print("Restaurant 'Penang Flavours' not found in the database.")

Restaurant 'Penang Flavours' found in the database.


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [80]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_query = {"BusinessType": "Restaurant/Cafe/Canteen"}

#just need BusinessID and BusinessType so _id: 0 will exclude it.
business_type_projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

business_types = establishments.find_one(business_type_query, business_type_projection)
pprint(business_types)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [81]:
# Update the new restaurant with the correct BusinessTypeID
business_type_id_to_update = "7"

In [82]:
# Confirm that the new restaurant was updated
update_result = establishments.update_one(
    {"BusinessName": "Penang Flavours"}, 
    {"$set": {"BusinessTypeID": business_type_id_to_update}}
)

business_type_id_to_update_check = establishments.find_one({"BusinessTypeID": "7"})
if business_type_id_to_update_check:
    print("ID is 7")
else:
    print("Not Updated")

ID is 7


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [83]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(dover_count)

0


In [84]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})

In [85]:
# Check if any remaining documents include Dover
remaining_dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(remaining_dover_count)

0


In [86]:
# Check that other documents remain with 'find_one'
if establishments.find_one():
    print("Other documents still exist.")
else:
    print("No documents found.")

Other documents still exist.


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [87]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {},  # This empty query matches all documents
    [{'$set': {
                'geocode.latitude': {
                    '$toDecimal': '$geocode.latitude'
                },
                'geocode.longitude': {
                    '$toDecimal': '$geocode.longitude'
                }
              }
    }]
)

UpdateResult({'n': 38788, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [88]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [89]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {'RatingValue': {'$not': {'$in': non_ratings}}},
    [
        {'$set': {
            'RatingValue': {'$convert': {'input': '$RatingValue', 'to': 'int', 'onError': ''}}
        }}
    ]
)

UpdateResult({'n': 38788, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [90]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments.find_one()

latitude_type = type(sample_document['geocode']['latitude'])
longitude_type = type(sample_document['geocode']['longitude'])
rating_value_type = type(sample_document['RatingValue'])

print(f"Latitude type: {latitude_type}")
print(f"Longitude type: {longitude_type}")
print(f"RatingValue type: {rating_value_type}")

Latitude type: <class 'bson.decimal128.Decimal128'>
Longitude type: <class 'bson.decimal128.Decimal128'>
RatingValue type: <class 'int'>
